## Examples for demonstrating different classification techniques

In [1]:
%matplotlib widget

import numpy as np
import xarray as xa
import matplotlib.pyplot as plt
from skimage import exposure # We are going to rescale the image
import sklearn as skl
from sklearn.cluster import KMeans # Unsupervised learning
from sklearn.neural_network import MLPClassifier # Supervised learning
from matplotlib.colors import ListedColormap # For colourmap
from matplotlib import patches

We have now imported some libraries and will load some Sentinel 2 OpenDAP NetCDF
data from https://satellittdata.no

In [2]:
url = 'https://nbstds.met.no/thredds/dodsC/NBS/S2A/2021/07/14/S2A_MSIL1C_20210714T105031_N0301_R051_T33WWR_20210714T130146.nc'
dset = xa.open_dataset(url+'#fillmismatch') # opening OpenDAP NetCDF dataset

Having opened the dataset we can how have a look at the header

In [3]:
print(dset)

<xarray.Dataset>
Dimensions:                                             (dimension_rgb: 3, time: 1, x: 10980, y: 10980)
Coordinates:
  * time                                                (time) datetime64[ns] ...
  * x                                                   (x) int32 499980 ... ...
  * y                                                   (y) int32 7600020 ......
    lat                                                 (y, x) float32 ...
    lon                                                 (y, x) float32 ...
Dimensions without coordinates: dimension_rgb
Data variables: (12/71)
    TCI                                                 (dimension_rgb, y, x) float32 ...
    UTM_projection                                      int32 ...
    S2_Level_1C_Product_Metadata                        |S64 ...
    INSPIRE_Metadata                                    |S64 ...
    S2_Level_1C_Datastrip1_Metadata                     |S64 ...
    Format_OLQC_Report_Datastrip1_InformationData  

Notice that it contains coordinates, variables, and attributes. We can for
instance take a closer look at the time coordinate

In [4]:
print(dset.time)

<xarray.DataArray 'time' (time: 1)>
array(['2021-07-14T10:50:31.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2021-07-14T10:50:31
Attributes:
    long_name:  reference time of satellite image


and the B2 data variable

In [5]:
print(dset.B2)

<xarray.DataArray 'B2' (time: 1, y: 10980, x: 10980)>
[120560400 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2021-07-14T10:50:31
  * x        (x) int32 499980 499990 500000 500010 ... 609750 609760 609770
  * y        (y) int32 7600020 7600010 7600000 ... 7490250 7490240 7490230
    lat      (y, x) float32 ...
    lon      (y, x) float32 ...
Attributes:
    units:                  1
    grid_mapping:           UTM_projection
    standard_name:          toa_bidirectional_reflectance
    long_name:              Reflectance in band B2
    bandwidth:              65
    bandwidth_unit:         nm
    wavelength:             490
    wavelength_unit:        nm
    solar_irradiance:       1959.66
    solar_irradiance_unit:  W/m2/um
    _ChunkSizes:            [ 1 32 32]


**What is B2?**

We can plot B2 to see it better

In [6]:
# East - west
xmin = 9000
xmax = -1

# North - south
ymin = 0
ymax = 2000

plt.figure()
view =dset.B2[0,ymin:ymax,xmin:xmax]
view.plot(cmap='Greys',vmax=1000)
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
print(dset.TCI)

<xarray.DataArray 'TCI' (dimension_rgb: 3, y: 10980, x: 10980)>
[361681200 values with dtype=float32]
Coordinates:
  * x        (x) int32 499980 499990 500000 500010 ... 609750 609760 609770
  * y        (y) int32 7600020 7600010 7600000 ... 7490250 7490240 7490230
    lat      (y, x) float32 ...
    lon      (y, x) float32 ...
Dimensions without coordinates: dimension_rgb
Attributes:
    long_name:     TCI RGB from B4, B3 and B2
    grid_mapping:  UTM_projection
    units:         1
    _ChunkSizes:   [   1 3660 3660]


In [8]:
extent = np.asarray(
            [dset.lon[ymin,xmin].data, dset.lon[ymin,xmax].data,
            dset.lat[ymax,xmax].data, dset.lat[ymin,xmin].data]
         ) # To get the axis labels correct we define an extent

#print('lon and lat',(extent)) # These are the latitudes and longitudes
n=0
# Reading in some parts of the r b g bands
b = dset.B2[n,ymin:ymax,xmin:xmax]/float(dset.attrs['QUANTIFICATION_VALUE'])
g = dset.B3[n,ymin:ymax,xmin:xmax]/float(dset.attrs['QUANTIFICATION_VALUE'])
r = dset.B4[n,ymin:ymax,xmin:xmax]/float(dset.attrs['QUANTIFICATION_VALUE'])

P0, P1 = (0, 0.15) # These are the percentages we are going to rescale on 

# Rescaling the exposures using scikit-image 
r = exposure.rescale_intensity(r,in_range=(P0,P1))
g = exposure.rescale_intensity(g,in_range=(P0,P1))
b = exposure.rescale_intensity(b,in_range=(P0,P1))

# Making the RGB image
rgb = np.dstack((r,g,b))

# Clipping it
rgb[rgb>1]=1.0


# Plotting the RGB image
plt.figure(figsize=(7,6))
plt.imshow(rgb, interpolation='none', aspect='auto', extent = extent)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Normalized difference vegetation index (NDVI)
Is a measure for how much vegitation there is in a pixel. More information can be found in the book (10.6), 
https://custom-scripts.sentinel-hub.com/sentinel-2/ndvi/
and
https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index

It is given by:
$\text{NDVI} = \frac{\textrm{NIR}-\textrm{RED}}{\textrm{NIR}+\textrm{RED}}$

which for Sentinel-2 becomes:

$\text{NDVI} = \frac{\textrm{B8}-\textrm{B4}}{\textrm{B8}+\textrm{B4}}$

We are now going to test it, though we are using L1C data. Normally one would use L2A

In [9]:
ndvi = (dset.B8[n,ymin:ymax,xmin:xmax] -dset.B4[n,ymin:ymax,xmin:xmax])/(dset.B8[n,ymin:ymax,xmin:xmax] +dset.B4[n,ymin:ymax,xmin:xmax])
plt.figure(figsize=(7,6))
plt.imshow(ndvi, interpolation='none', aspect='auto', extent = extent)
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Normalized difference water index (NDWI)
Measures how much water there is in the pixel as part of a water body.  More information can be found at:
https://custom-scripts.sentinel-hub.com/sentinel-2/ndwi/
og 
https://en.wikipedia.org/wiki/Normalized_difference_water_index


It is given by:
$\text{NDWI} = \frac{\textrm{GREEN}-\textrm{NIR}}{\textrm{GREEN}+\textrm{NIR}}$


which for Sentinel-2 becomes:

$\text{NDWI} = \frac{\textrm{B3}-\textrm{B8}}{\textrm{B3}+\textrm{B8}}$

We are now going to test it, though we are using L1C data. Normally one would use L2A

In [10]:
ndwi = (dset.B3[n,ymin:ymax,xmin:xmax] -dset.B8[n,ymin:ymax,xmin:xmax])/(dset.B3[n,ymin:ymax,xmin:xmax] +dset.B8[n,ymin:ymax,xmin:xmax])
plt.figure(figsize=(7,6))
plt.imshow(ndwi, interpolation='none', aspect='auto', extent = extent)
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Unsupervised learning

This is learning where we as the computer to classify the area based on a given model. In this case we are going to use [scikit-learn](https://scikit-learn.org/stable/)
and the method [K-means](https://scikit-learn.org/stable/modules/clustering.html#k-means) to cluster data. It is based on the minimising the variance in the cluster, though something called its intertia. The link above has an excellent explanation.




In [11]:
  # We are first going to make the dataset, which here is going to be rgb + NIR

rgbn = np.dstack((dset.B4[n,ymin:ymax,xmin:xmax],dset.B3[n,ymin:ymax,xmin:xmax],
      dset.B2[n,ymin:ymax,xmin:xmax],dset.B8[n,ymin:ymax,xmin:xmax]))

w, h, d = original_shape = tuple(rgbn.shape)

rgbn_2d = np.reshape(rgbn,(w*h,d))
print(rgbn_2d.shape)
# We select a small subset to fit on:
rgbn_sample = skl.utils.shuffle(rgbn_2d, random_state=0, n_samples=1_000)

# We are now going to ask for a certain number of clusters
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rgbn_sample)

# Predict the labels for the whole image 
# (this could also be a new image, and can be reused on multiple images)

labels = kmeans.predict(rgbn_2d)
print(labels.shape)

# Show the results
plt.figure(figsize=(7,6))
plt.imshow(labels.reshape(w,h), interpolation='none', aspect='auto', extent = extent)
plt.show()

(3958000, 4)
(3958000,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Supervised learning

In contrast to unsupervised learning, in supervised learning we need to teach the algorithm what a given class looks like. 
We have a range of different algorithms to choose from, and it would be to much to go through them all. 
Here we choose to use a neural network, specifically "Multi-layer Perceptron" from scikit-learn. You can read more about it at [Neural network models (supervised)](https://scikit-learn.org/stable/modules/neural_networks_supervised.html)

There are much faster implmentations neural networks in deep learning architectures, but these require the use of GPUs and dedicated software (for instance [TensorFLow](https://www.tensorflow.org/) ). SNAP also has built in modules for supervised learning.



In [12]:
# We first need to define som areas to learn from. We are going to look at classifying into 
# four different types and we will be using the same dataset (rgbn) as for unsupervised classification

wx=[500, 700]
wy=[250, 500]

water = rgbn[wx[0]:wx[1],wy[0]:wy[1],:]
water = np.reshape(water,(water.shape[0]*water.shape[1],d)) # making it 2d

tx=[560, 660]
ty=[860, 910]

trees = rgbn[tx[0]:tx[1],ty[0]:ty[1],:]
trees = np.reshape(trees,(trees.shape[0]*trees.shape[1],d)) # making it 2d

cx=[1185, 1225]
cy=[690, 740]

clouds = rgbn[cx[0]:cx[1],cy[0]:cy[1],:]
clouds = np.reshape(clouds,(clouds.shape[0]*clouds.shape[1],d)) # making it 2d

bx=[943, 970]
by=[920, 945]

build = rgbn[bx[0]:bx[1],by[0]:by[1],:]
build = np.reshape(build,(build.shape[0]*build.shape[1],d)) # making it 2d



def plot_rect(xlim, ylim, text, col):
    ax = plt.gca() # get current axis
    ax.add_patch(patches.Rectangle((xlim[0], ylim[0]), ylim[1]-ylim[0], xlim[1]-xlim[0], edgecolor=col, fill=False))
    ax.annotate(text,(xlim[1]+10, ylim[0]+(ylim[1]-ylim[0])/2), color=col, ha='left', va='center')
    

# Show where the rectangles are
plt.figure(figsize=(7,6))
plt.imshow(rgb, interpolation='none', aspect='equal')#, extent = extent) # Plotting it in x y coordinates and not lat lon
plot_rect(wx, wy, "Water", "red")
plot_rect(tx, ty, "Trees", "white")
plot_rect(cx, cy, "Clouds", "green")
plot_rect(bx, by, "City", "black")
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# Now we are going to use these to train the algorithm
X = np.concatenate([water, trees, clouds, build])

#Assigning labels 0 = water, 1 = trees, 2 = clouds, 3 = city
y = np.concatenate([
    np.zeros(water.shape[0]),
    np.ones(trees.shape[0]),
    np.ones(clouds.shape[0]) + 1,
    np.ones(build.shape[0]) + 2])
# 
print(X.shape,y.shape)
# Initialise the neural network
clf = MLPClassifier() # We are using default values
clf.fit(X, y)

# How many layers and outputs are we using:
print(clf.n_layers_, clf.n_outputs_)

labels_nn = clf.predict(rgbn_2d)
print(labels_nn.shape)




(57675, 4) (57675,)
3 4
(3958000,)


In [16]:

# Assigning colours to the different labels
col_dict={0:"blue", # Sea
          1:"green", # Trees
          2:"white", # Clouds
          3:"black"} # City


cm = ListedColormap([col_dict[x] for x in col_dict.keys()])

plt.figure(figsize=(7,6))
plt.imshow(labels_nn.reshape(w,h), interpolation='none', aspect='equal', cmap=cm)#, extent = extent,cmap=cm)
cbar = plt.colorbar( orientation='horizontal',ticks=[3/8,2*3/4-3/8,3*3/4-3/8,4*3/4-3/8])
cbar.ax.set_xticklabels(['Sea', 'Trees', 'Clouds', 'Houses'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …